# Advanced Lane Finding

by __Vikas Hanasoge Nataraja__



### Step 1: Import all libraries and packages

In [8]:
import numpy as np
import os
import sys
# using the following line because OpenCV is installed in a different directory on my PC
# comment out this line if that is not the case on your PC
sys.path.append('C:/ProgramData/Anaconda3/Lib/site-packages')
import cv2
import matplotlib.image as mpimg
import glob
import matplotlib.pyplot as plt

# Import everything needed to process video clip for later
from moviepy.editor import VideoFileClip
from IPython.display import HTML

### Step 2: Algorithm pipeline

#### 2.1 Define a class with a method to fit the line of the lane

In [9]:
class LineClass():
    def __init__(self):
        # polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        # Initialize all others not carried over between first detections
        self.reset()

    def reset(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # recent polynomial coefficients
        self.recent_fit = []
        # polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        # difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        # x values for detected line pixels
        self.all_x = None  
        # y values for detected line pixels
        self.all_y = None
        # counter to reset after 5 iterations if issues arise
        self.counter = 0

    def fit_line(self, x_points, y_points, first_try=True):
        """
        Fit a second order polynomial to the line because second order describes a curve
        """
         
        n = 5
        self.current_fit = np.polyfit(y_points, x_points, 2)
        self.all_x = x_points
        self.all_y = y_points
        self.recent_fit.append(self.current_fit)
        if len(self.recent_fit) > 1:
            self.diffs = (self.recent_fit[-2] - self.recent_fit[-1]) / self.recent_fit[-2]
        self.recent_fit = self.recent_fit[-n:]
        self.best_fit = np.mean(self.recent_fit, axis = 0)
        line_fit = self.current_fit
        self.detected = True
        self.counter = 0


        return line_fit


#### 2.2 Calibration

Calibrate the camera using the chessboard images provided

In [10]:
def camera_calibration(images_chess):
    '''
    Perform camera calibration.
    '''
    
    calibration_images = []
    # Load in the chessboard calibration images to a list
    for fname in images_chess:
        img = mpimg.imread(fname)
        calibration_images.append(img)
      

    # Prepare object points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    # Arrays for later storing object points and image points
    objpoints = []
    imgpoints = []
    outputimages = []
    originalimages = []

    # Iterate through images for their points
    for image in calibration_images:
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            img_points = (cv2.drawChessboardCorners(img.copy(), (9, 6), corners, ret))
            outputimages.append(img_points)
            originalimages.append(img)
    
    # debug
    #cv2.imshow('original image',calibration_images[0])       
    #cv2.imshow('calib image',outputimages[0])
    #if (cv2.waitKey(0) & 0xFF) == 27:
    #    cv2.destroyAllWindows()
    #cv2.set_title('chess points')
    #cv2.imwrite('output_images/calibrated_image.png',img_points)
    #cv2.imwrite('output_images/original_image.png',img)


    # Return camera calibration parameters
    ret, matrix, distance, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, 
                                                               gray.shape[::-1], None,None)
                                                       

    return matrix, distance

#### 2.3 Apply Sobel filter and generate a binary image

In [11]:
def pipeline(img, mat, dist, s_thresh=(125, 255), sx_thresh=(10, 100), 
             R_thresh = (200, 255), sobel_kernel = 3):
    """
    Pipeline to create binary image.
    Binary activation occurs where any two of the three (R and S color channels and Sobel filter) are activated.
    """
    distorted_img = np.copy(img)
    dst = cv2.undistort(distorted_img, mat, dist, None, mat)
    #print(type(dst))
    #cv2.imshow('undistorted image',dst)
    #cv2.imwrite('output_images/undistorted_image.png',dst)
    # Convert to HLS colorspace
    hls = cv2.cvtColor(dst, cv2.COLOR_RGB2HLS).astype(np.float)
   
    
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobelx - takes the derivative in x, absolute value,followed by rescaling
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobelx)
    sxbinary[(scaled_sobelx >= sx_thresh[0]) & (scaled_sobelx <= sx_thresh[1])] = 1
             
    
    # Consider red channel
    red = dst[:,:,0]

    # Threshold red color channel
    R_binary = np.zeros_like(red)
    R_binary[(red >= R_thresh[0]) & (red <= R_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # If two of the three are activated, activate in the binary image
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[((s_binary == 1) & (sxbinary == 1)) 
                     | ((sxbinary == 1) & (R_binary == 1))
                     | ((s_binary == 1) & (R_binary == 1))] = 1
    #cv2.imshow('binary image',s_binary)
    #cv2.imwrite('output_images/combined_binary_image.png',s_binary)
    #print(s_binary.shape)
    #print('-----------')
    #print(combined_binary.shape)
    #if (cv2.waitKey(0) & 0xFF) == 27:
    #    cv2.destroyAllWindows()
    return combined_binary


#### 2.4 Undistort the images 
Using calibration performed in the earlier cell, undistort the image.
This transform also changes the view to bird's eye perspective

In [ ]:
def birds_eye_transform(img, mat, dist):
    """
    First undistort the image using the calibration from earlier.
    Next, using defined source image points and destination points,
    transform the image as if the road was viewed from above,
    like a bird would see. Returns the birds eye image and transform matrix.
    """
    
    # Put the image through the pipeline to get the binary image
    binary_img = pipeline(img, mat, dist)

    # Grab the image shape
    img_size = (binary_img.shape[1], binary_img.shape[0])

    # Source points - defined area of lane line edges
    src = np.float32([[690,450],[1110,img_size[1]],[175,img_size[1]],[595,450]])

    # 4 destination points to transfer
    offset = 300 # offset for dst points
    dst = np.float32([[img_size[0]-offset, 0],[img_size[0]-offset, img_size[1]],
                      [offset, img_size[1]],[offset, 0]])
    
    # Use cv2.getPerspectiveTransform() to get the transform matrix
    transform_matrix = cv2.getPerspectiveTransform(src, dst)
    
    # Use cv2.warpPerspective() to warp the image to a top view
    top_view = cv2.warpPerspective(binary_img, transform_matrix, img_size)
    #cv2.imshow('bird\'s eye view',top_view)
    #cv2.imwrite(cv2.imwrite('output_images/top_view.png',top_view))
    
    #if (cv2.waitKey(0) & 0xFF) == 27:
    #    cv2.destroyAllWindows()
    return top_view, transform_matrix


#### 2.5 Sliding window

Use a sliding window to check for non-zero pixels.
If above a certain threshold, re-center the window to their mean.
Also check if window is within the image bounds.

In [ ]:
def sliding_window(x_present, margin, minpix, nonzerox, nonzeroy, 
                   win_y_low, win_y_high, window_max, counter, side):
    """
    Creates a window, then checks for non-zero value pixels within it.
    If there are more than minpix, the window gets re-centered to their mean.
    Lastly, if at least five windows have been used, check whether we have
    hit the outside of the image.
    """
    
    # Get the window boundaries
    win_x_low = x_present - margin
    win_x_high = x_present + margin
    
    # Identify the nonzero pixels in x and y within the window
    nonzero_indices = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) 
                 & (nonzerox >= win_x_low) 
                 & (nonzerox < win_x_high)).nonzero()[0]
    # If you found > minpix pixels, recenter next window on their mean position
    if len(nonzero_indices) > minpix:
        x_present = np.int(np.mean(nonzerox[nonzero_indices]))
    if counter >= 5:
        if win_x_high > window_max or win_x_low < 0:
            if side == 'left':
                left_tracker = False
            else:
                right_tracker = False

    return nonzero_indices, x_present


#### 2.6 Histogram
Using histograms, I can see where the binary triggers occur. This is follwed by estimating where the actual lane lines are.

In [ ]:
def initial_lines_histogram(img, mat, dist):
    """
    First Lines uses the birds eye image from above,
    creates a histogram of where the binary activations occur,
    and uses sliding windows along the peak areas to estimate
    where the lane lines are.
    """
    
    binary_warped, perspective_M = birds_eye_transform(img, mat, dist)
    
    # Histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 35
    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set the width of the windows +/- margin
    margin = 100
    
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    left_tracker = True
    right_tracker = True
    counter = 0

    # Step through the windows one by one
    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        window_max = binary_warped.shape[1]
        if left_tracker == True and right_tracker == True:
            nonzero_left_inds, leftx_current = sliding_window(leftx_current, margin, minpix, nonzerox, nonzeroy, 
                                                           win_y_low, win_y_high, window_max, counter, 'left')
            nonzero_right_inds, rightx_current = sliding_window(rightx_current, margin, minpix, nonzerox, nonzeroy, 
                                                             win_y_low, win_y_high, window_max, counter, 'right')
            # Append these indices to the lists
            left_lane_inds.append(nonzero_left_inds)
            right_lane_inds.append(nonzero_right_inds)
            counter += 1
        elif left_tracker == True:
            nonzero_left_inds, leftx_current = sliding_window(leftx_current, margin, minpix, nonzerox, nonzeroy, 
                                                           win_y_low, win_y_high, window_max, counter, 'left')
            # Append these indices to the list
            left_lane_inds.append(nonzero_left_inds)
        elif right_tracker == True:
            nonzero_right_inds, rightx_current = sliding_window(rightx_current, margin, minpix, nonzerox, nonzeroy, 
                                                             win_y_low, win_y_high, window_max, counter, 'right')
            # Append these indices to the list
            right_lane_inds.append(nonzero_right_inds)
        else:
            break

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each line
    left_fit = left_line.fit_line(leftx, lefty, True)
    right_fit = right_line.fit_line(rightx, righty, True)

    

#### 2.7 Fit function
Here I am using a second order polynomial because they estimate curves (think equation for a circle or ellipse - both are second order)

In [ ]:
def second_order_polynomial(line, value):
    """
    Simple function being used to help calculate distance from center.
    Used to find the base of the line at the bottom of the image.
    """
    a = line[0]
    b = line[1]
    c = line[2]
    # second order equation a*x**2 + b*x + c
    formula = (a*value**2)+(b*value)+c

    return formula

#### 2.8 Draw the lane lines
After detections occur, estimate the curvature of the lane and draw the actual lane lines

In [ ]:

def draw_lane_lines(img, matrix, dist):
    '''
    Draw Lines will first check whether the lines are detected.
    If not, go back up to `initial_lines_histogram`. If they are, we do not have to search
    the whole image for the lines. We can then draw the lines,
    as well as detect where the car is in relation to the middle of the lane,
    and what type of curvature it is driving at.
    '''
    # Pull in the image
    binary_warped, perspective_M = birds_eye_transform(img, matrix, dist)

    # Check if lines were last detected; if not, re-run
    if left_line.detected == False or right_line.detected == False:
        initial_lines_histogram(img, matrix, dist)

    # Set the fit as the current fit for now
    left_fit = left_line.current_fit
    right_fit = right_line.current_fit

    # Again, find the lane indicators
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) 
                      & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) 
                       & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Set the x and y values of points on each line
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Fit a second order polynomial to each again.
    left_fit = left_line.fit_line(leftx, lefty, False)
    right_fit = right_line.fit_line(rightx, righty, False)

    # Generate x and y values for plotting
    fity = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    fit_leftx = left_fit[0]*fity**2 + left_fit[1]*fity + left_fit[2]
    fit_rightx = right_fit[0]*fity**2 + right_fit[1]*fity + right_fit[2]

    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)

    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([fit_leftx-margin, fity]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([fit_leftx+margin, fity])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([fit_rightx-margin, fity]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([fit_rightx+margin, fity])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Calculate the pixel curve radius
    y_eval = np.max(fity)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(left_line.all_y*ym_per_pix, 
                             left_line.all_x*xm_per_pix, 2)
    right_fit_cr = np.polyfit(right_line.all_y*ym_per_pix, 
                              right_line.all_x*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    avg_rad = round(np.mean([left_curverad, right_curverad]),0)
    
    # display the text annotated on the video
    rad_text = 'Radius of Curvature = {}(m)'.format(avg_rad)

    # Calculating middle of the image, aka where the car camera is
    middle_of_image = img.shape[1] / 2
    car_position = middle_of_image * xm_per_pix

    # Calculating middle of the lane
    left_line_base = second_order_polynomial(left_fit_cr, img.shape[0] * ym_per_pix)
    right_line_base = second_order_polynomial(right_fit_cr, img.shape[0] * ym_per_pix)
    lane_mid = (left_line_base+right_line_base)/2

    # Calculate distance from center and list differently based on left or right
    dist_from_center = lane_mid - car_position
    
    # display the distance from the center of the lane on the video, annotated
    if dist_from_center >= 0:
        center_text = '{} meters left of center'.format(round(dist_from_center,2))
    else:
        center_text = '{} meters right of center'.format(round(-dist_from_center,2))
        
    # List car's position in relation to middle on the image and radius of curvature
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, center_text, (10,50), font, 1,(255,255,255),2)
    cv2.putText(img, rad_text, (10,100), font, 1,(255,255,255),2)

    # Invert the transform matrix from birds_eye (to later make the image back 
    #   to normal below)
    invertedM = np.linalg.inv(perspective_M)

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([fit_leftx, fity]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([fit_rightx, fity])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix
    newwarp = cv2.warpPerspective(color_warp, invertedM, (img.shape[1], img.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)

    return result

In [ ]:
def process_image(image, matrix, distance):
    """
    This processes through everything above.
    The result returns the car position, lane lines drawn on the image and lane curvature
    """
    result = draw_lane_lines(image, matrix, distance)
    #print(result)
    return result

### Step 3: Main function


In [ ]:
def main():
    #  location of calibration images
    image_folder = glob.glob('camera_cal/*.jpg')

    # Calibrate camera and return calibration data
    mtx, dist = camera_calibration(image_folder)

    # name of output video
    video_output = 'out_video.mp4'

    readVideo = VideoFileClip('project_video.mp4')

    # NOTE: this function expects color images
    videoCar = readVideo.fl_image(lambda image: process_image(image, mtx, dist))
    videoCar.write_videofile(video_output, audio=False)



### Step 4: Run the program

In [ ]:
if __name__ == '__main__':
    # Set the class lines equal to the variables used above
    left_line = LineClass()
    right_line = LineClass()
    main()